# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [13]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [14]:
# your code here
df=pd.read_csv("Orders.csv")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
#SUB PROBLEM 1
m=df.groupby("CustomerID").sum()["amount_spent"]
display(m.head())

rang=[np.percentile(m.values,75),np.percentile(m.values,95)]

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
Name: amount_spent, dtype: float64

In [16]:
#SUB PROBLEM 2

m2=m.to_frame().reset_index()
def enRango(x):
    if (float(x)>=rang[0]) and (float(x)<=rang[1]): return x
    return 0
def enRango2(x):
    if (float(x)>rang[1]): return x
    return 0
m2["Preferred"]=m2["amount_spent"].apply(enRango)
m2["VIP"]=m2["amount_spent"].apply(enRango2)
display(m2.head())
vip=m2[m2["VIP"]>0][["CustomerID","amount_spent"]]
preferred=m2[m2["Preferred"]>0][["CustomerID","amount_spent"]] #filtramos los valores que no queremos
print(rang)
display(preferred.head())
display(vip.head())

,CustomerID,amount_spent,Preferred,VIP
0,12346,77183.60,0.00,77183.6
1,12347,4310.00,4310.00,0.0
2,12348,1797.24,1797.24,0.0
3,12349,1757.55,1757.55,0.0
4,12350,334.40,0.00,0.0


[1661.6400000000003, 5840.181999999983]


,CustomerID,amount_spent
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43


,CustomerID,amount_spent
0,12346,77183.60
10,12357,6207.67
12,12359,6372.58
50,12409,11072.67
55,12415,124914.53


In [22]:
#SUB PROBLEM 3



In [23]:

df3=df.copy()
id_vip=vip["CustomerID"].values
id_pr=preferred["CustomerID"].values
def func(x):
    if int(x) in id_vip: return "VIP"
    elif int(x) in id_pr: return "Preferred"
    return "NULO"

df3["Group"]=df["CustomerID"].apply(func)
df3["Group"].value_counts()


Preferred    151781
NULO         141659
VIP          104484
Name: Group, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
q=df3[df3["Group"]!="NULO"]
df4=df3[df3["Group"]!="NULO"]
q=pd.get_dummies(q["Group"])
df4[["VIP","Preferred"]]=q[["VIP","Preferred"]]
df4.head()
#q.groupby("Country").sum()q[""]

/Users/usuario/Library/Python/3.6/lib/python/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Group,VIP,Preferred
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred,0,1
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,0,1
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred,0,1
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,0,1
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,0,1


In [30]:
df4.groupby("Country").sum()["VIP"].to_frame().reset_index().sort_values(by="VIP",axis=0,ascending=False).head(1)

,Country,VIP
26,United Kingdom,84185.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [32]:
# your code here
df4.groupby("Country").sum()[["VIP","Preferred"]].reset_index().sort_values(by=["VIP","Preferred"],axis=0,ascending=False).head(1)

,Country,VIP,Preferred
26,United Kingdom,84185.0,137450.0
